## **Installing Scrapy**

In [ ]:
!cd content/

In [ ]:
!conda install -c conda-forge scrapy
!pip install Scrapy

In [ ]:
!scrapy startproject scarpyproject

In [ ]:
%cd scarpyproject/scarpyproject/spiders

In [ ]:
!touch group7_spider.py

## **Make a Spider**

#### **Note:** Below is the spider script which needs to be written in `group7_spider.py` **NOT HERE**!

In [ ]:
import scrapy

class PolitifactSpider(scrapy.Spider):
    num_page = 0
    name = 'politifact'
    start_urls = [
        'https://www.politifact.com/factchecks/'
    ]

    def parse(self, response):
        items = ScarpyprojectItem()

        all_div_list_items = response.css('li.o-listicle__item')

        for list_item in all_div_list_items:
            link = list_item.css('.m-statement__quote a::attr(href)').get()

            yield response.follow(link, callback=self.parseInner)

        if self.num_page == 0:
            next_page = response.css('li.m-list__item a.c-button--hollow::attr(href)').get()
        else:
            next_page = response.css('.m-list__item+ .m-list__item .c-button--hollow::attr(href)').get()

        if next_page is not None:
            if self.num_page < 1:
                self.num_page += 1
                yield response.follow(next_page, callback=self.parse)

    def parseInner(self, response):
        items = ScarpyprojectItem()
        title = response.css('article.m-statement--is-xlarge div.m-statement__quote::text').extract()
        description = response.css('.m-textblock p::text').extract()
        published_at = response.css('.m-author__date::text').extract()
        authors = response.css('.u-color--chateau a::text').extract()
        tags = response.css('.c-tag span::text').extract()
        short_description = response.css('.short-on-time p::text').extract()
        if(short_description is None):
            short_description = response.css('div.short-on-time li.selectorgadget_selected::text').extract()
            if (short_description is None):
                short_description = response.css('.short-on-time li::text').extract()
                if (short_description is None):
                    short_description = response.css('.short-on-time span::text').extract()

        image = response.css('img.lozad::attr(data-src)').extract()
        images = response.css('div.artembed img::attr(src)').extract()
        status = response.css('article.m-statement--is-xlarge img.c-image__original::attr(alt)').get()

        items['title'] = title
        items['link'] = response.request.url
        items['description'] = description
        items['published_at'] = published_at
        items['authors'] = authors
        items['tags'] = tags
        items['short_description'] = short_description
        items['image'] = image
        items['images'] = images
        items['status'] = status

        yield items

In [ ]:
%cd ..

## **Run the Spider**

In [ ]:
!pwd #it should be /content/tutorial/tutorial

In [ ]:
%cd /content/scarpyproject/scarpyproject

In [ ]:
!scrapy crawl PolitifactSpider -o result.csv 

In [ ]:
from google.colab import files

files.download('./result.csv') 



# Add 1 Second Delay For Polity

In [ ]:
from google.colab import files

files.download('./settings.py') 
#search for DOWNLOAD_DELAY in the file and oncomment it and change the time to 1 second and save file

In [ ]:
!pwd #it should be /content/tutorial/tutorial

In [ ]:
!ls
#it shoud be settings.py in the list

In [ ]:
!rm settings.py
#then from the inner "tutorial" folder options select upload and select modified settings.py and run the crawler